# Texts + Embeddings 

In [2]:
# processing
import re
import os
from langchain_core import documents
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# neo4j
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector

# llm / gigachat
from langchain_community.embeddings import GigaChatEmbeddings

### Text Splitter

In [9]:
class CustomTextSplitter:
    def __init__(self, target_chunk_size=1000):
        
        self.target_chunk_size = target_chunk_size
        self.source = ''
        self.chunks = []
        self.current_chunk = ''
        self.dcts = []
        self.bullet_name = ''
        self.prev_bullet_name = ''
        self.mode_list = ['bullet', 'line', 'space']
        self.sep_dict = {'line': '\n', 'space': ' '}
        self.link_dict = {'bullet': '\n', 'line': '\n', 'space': ' '}
        
    def is_bullet(self, line):
        # Проверка, начинается ли строка с буллита
        return re.match(r'(\d+(\.\d+)*\.\d+)\.?', line.strip())
    
    def make_chunks(self, input_text, mode):
        link = self.link_dict[mode] 
        if len(input_text) == 0:
            return
        if len(self.current_chunk) > 0:
            # Если текущий чанк не пуст, проверяем, поместится ли туда текущий фрагмент
            new_chunk = self.current_chunk + link + input_text
            if len(new_chunk) <= self.target_chunk_size and mode != 'bullet':
                    self.current_chunk = new_chunk
            else:
                # Сначала сохраняем текущий чанк, потом 
                # в текущий чанк записываем новый фрагмент
                self.chunks.append(self.current_chunk)
                dct = documents.base.Document(self.current_chunk)
                dct.metadata = {'source': self.source}
                # Записываем имя буллита чанка в метаданные
                if mode == 'bullet':
                    dct.metadata['bullet'] = self.prev_bullet_name
                else:
                    dct.metadata['bullet'] = self.bullet_name
                self.dcts.append(dct)
                self.current_chunk = ''
                if len(input_text) <= self.target_chunk_size:
                    self.current_chunk = input_text
                else:
                    # Текст не помещается в чанк целиком. Тогда определяем параметры нового 
                    # разбиения и вызываем рекурсивно этот же метод.
                    next_mode, separator = self.prepare_for_new_split(mode)
                    if next_mode is None:
                        return
                    else:
                        for item in input_text.split(separator):
                            self.make_chunks(item, next_mode)
                        # После завершения разбиения сохраняем текущий чанк, если он не пуст.    
                        if self.current_chunk != '':
                            self.chunks.append(self.current_chunk)
                            dct = documents.base.Document(self.current_chunk)
                            # В текущей реализации в метаданные пишется имя последнего буллита чанка
                            dct.metadata = {'source': self.source,'bullet': self.bullet_name}
                            self.dcts.append(dct)
                            self.current_chunk = ''
        elif len(input_text) <= self.target_chunk_size:
                self.current_chunk = input_text
        else:
            # Текст не помещается в чанк целиком. Тогда определяем параметры нового 
            # разбиения и вызываем рекурсивно этот же метод.            
            next_mode, separator = self.prepare_for_new_split(mode)
            if next_mode is None:
                return
            else:
                for item in input_text.split(separator):
                    self.make_chunks(item, next_mode)
                # После завершения разбиения сохраняем текущий чанк, если он не пуст. 
                if self.current_chunk != '':
                    self.chunks.append(self.current_chunk)
                    dct = documents.base.Document(self.current_chunk)
                    # Записываем имя буллита чанка в метаданные
                    dct.metadata = {'source': self.source,'bullet': self.bullet_name}
                    self.dcts.append(dct)
                    self.current_chunk = ''
    
    def prepare_for_new_split(self, current_mode):
        # Определяем параметры разбиения
            next_mode_index = self.mode_list.index(current_mode) + 1
            if next_mode_index < len(self.mode_list):
                next_mode = self.mode_list[next_mode_index]
                next_separator = self.sep_dict[next_mode]
                return next_mode, next_separator
            else:
                return None, None
            
    def split_text(self, text, source):
        bullet_chunks = []
        current_bullet_chunk = ''
        self.source = os.path.splitext(os.path.basename(source))[0]
        # Разбиваем исходный текст на куски, каждый из которых (кроме первого) 
        # начинается с буллита
        lines = text.split("\n")
        for line in lines:
            if len(line) > 0:
                if self.is_bullet(line):
                    if len(current_bullet_chunk) > 0:
                        bullet_chunks.append(current_bullet_chunk)
                    current_bullet_chunk = line      
                elif len(current_bullet_chunk) > 0:
                    current_bullet_chunk = current_bullet_chunk + '\n' + line
                else:
                    current_bullet_chunk = line     
        bullet_chunks.append(current_bullet_chunk)
        # Для каждого буллита сохраняем его имя (=номер) и вызываем метод разбивки на чанки
        for bc in bullet_chunks:
            if len(bc) > 0:
                self.prev_bullet_name = self.bullet_name
                if self.is_bullet(bc):
                    self.bullet_name = self.is_bullet(bc).group()
                else:
                    self.bullet_name = bc.split()[0]
                self.make_chunks(bc, 'bullet')
                
        return self.dcts

## Splitting Texts

In [109]:
path = 'Закупка у ЕдП Приложение 2.txt'

In [110]:
loader = TextLoader(path, encoding = 'UTF-8')
text_documents = loader.load()

In [111]:
splitter = CustomTextSplitter()
split_documents = splitter.split_text(text_documents[0].page_content, text_documents[0].metadata['source'])
split_documents[:3]

[Document(page_content='1. Организация закупок для оказания услуг по авторскому контролю за разработкой проектной документации', metadata={'source': 'Закупка у ЕдП Приложение 2', 'bullet': '1.'}),
 Document(page_content='1.1. При проведении закупок для оказания услуг по авторскому контролю за разработкой проектной документации при НМЦ договора менее 10 млн руб. без НДС Инициатор оформляет комплект документов в соответствии с шаблоном (Ш-03.03.02.01-04).', metadata={'source': 'Закупка у ЕдП Приложение 2', 'bullet': '1.1.'}),
 Document(page_content='1.1.1. Реестр планируемых в предстоящем квартале закупок формируется и согласовывается: − в КЦ: на уровне Заместителя генерального директора по профилю деятельности / Руководителя структурного подразделения прямого подчинения Генеральному директору ПАО «Компания 1», − в Обществе Компания 1: на уровне Руководителя Общества. ', metadata={'source': 'Закупка у ЕдП Приложение 2', 'bullet': '1.1.1.'})]

In [112]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(text_documents)

for chunk in split_documents:
    # Get the 'source' metadata
    source = chunk.metadata['source']
    # Remove the '.txt' extension
    source = source.replace('.txt', '')
    # Update the 'source' metadata
    chunk.metadata['source'] = source
    
split_documents[:3]

[Document(page_content='1. Организация закупок для оказания услуг по авторскому контролю за разработкой проектной документации\n1.1. При проведении закупок для оказания услуг по авторскому контролю за разработкой проектной документации при НМЦ договора менее 10 млн руб. без НДС Инициатор оформляет комплект документов в соответствии с шаблоном (Ш-03.03.02.01-04).\n1.1.1. Реестр планируемых в предстоящем квартале закупок формируется и согласовывается: − в КЦ: на уровне Заместителя генерального директора по профилю деятельности / Руководителя структурного подразделения прямого подчинения Генеральному директору ПАО «Компания 1», − в Обществе Компания 1: на уровне Руководителя Общества. \n1.1.2. На основании согласованного реестра закупок Инициатор в КЦ / Обществе Компания 1 осуществляет заключение договора с единственным поставщиком (подрядчиком, исполнителем).', metadata={'source': 'Закупка у ЕдП Приложение 2'}),
 Document(page_content='1.2. При проведении закупок для оказания услуг по ав

## DB Neo4J

In [3]:
# Neo4j
NEO4J_URI="neo4j+ssc://7c47bfc7.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="LeyLlAzlYwLgQD4EYylA9vBXNb7q-TdLVahd_64r3bg"
NEO4J_DATABASE="neo4j"

# GigaChat
LLM_SCOPE = "GIGACHAT_API_PERS"
LLM_AUTH ="NmNiNzcwZDgtMjI1MC00NWQ0LWJjMDEtNTQ1ZTQ3MmE4NDllOjRmNWVkYTUwLTQ3NjgtNGNkZC05M2Q0LTFiMWNmODZmOGMyOQ=="

In [4]:
embeddings = GigaChatEmbeddings(credentials=LLM_AUTH, verify_ssl_certs=False)

In [ ]:
import pickle

with open('collections.pkl', 'rb') as f:
    collections = pickle.load(f)

In [120]:
vectorestore = Neo4jVector.from_documents(collections,
                                          embeddings,
                                          url = NEO4J_URI,
                                          username = NEO4J_USERNAME,
                                          password = NEO4J_PASSWORD,
                                          node_label = "Bullet",
                                          index_name = "bullet",
                                          keyword_index_name = "word",
                                          text_node_property = "text",
                                          embedding_node_property = "embedding",
                                          search_type = 'hybrid',
                                          create_id_index = True,
                                         )

In [7]:
vectorestore = Neo4jVector.from_existing_index(embeddings,
                                               url = NEO4J_URI,
                                               username = NEO4J_USERNAME,
                                               password = NEO4J_PASSWORD,
                                               node_label = "Bullet",
                                               index_name = "bullet",
                                               keyword_index_name = "word",
                                               text_node_property = "text",
                                               search_type = "hybrid")

In [8]:
print(vectorestore.index_name)
print(vectorestore.node_label)
print(vectorestore.keyword_index_name)
print(vectorestore.embedding_node_property)

bullet
Bullet
word
embedding
